In [1]:
import pandas as pd
import numpy as np
import json
import statistics
import math

In [2]:
import os

In [3]:
#Opening up the directory
directory = "/Users/nishantacharya/Desktop/BIRAFFE2/BIRAFFE2-games/BIRAFFE2-games"

In [4]:
test = os.listdir(directory)

In [5]:
File = directory+test[0]

In [6]:
gg = directory+"/"+test[0]

In [7]:
os.listdir(gg)

['SUB200-Level03_Log.json',
 'SUB200-Level02_Log.json',
 'SUB200-Level01_MapLog.json',
 'SUB200-Level02_BlockEvents.json',
 'SUB200-Level01_Log.json']

In [8]:
data = {}

In [9]:
#Collecting the data
for file in os.listdir(directory):
    if 'SUB' in file:
        sub_name = file[3:]
        data[sub_name] = None
        new_d = directory + '/' + file
        for new_f in os.listdir(new_d):
            if "Level01_Log" in new_f:
                jsn = new_d + "/" + new_f
                with open(jsn, 'r') as json_file:
                    json_load = json.load(json_file)
                
                
                # hori,vert,death,shoot,hit,collected money,collected health,sound id, sound stamp, 
                #hori min,vert min,hori maz,vert mazz
                hori_val = []
                vert_val = []
                death_val = []
                shoot_val = []
                hit_val = []
                mny = []
                hlth = []
                sndID = []
                sndTime = []
                hori_min = []
                vert_min = []
                hori_max = []
                vert_max = []
                timestamp = []
                
                
                for i in range(len(json_load)):
                    hori_val.append(json_load[i]['x'])
                    vert_val.append(json_load[i]['y'])
                    timestamp.append(json_load[i]['timestamp'])
                    death_val.append(json_load[i]['deathCount'])
                    shoot_val.append(json_load[i]['shootsCounter'])
                    hit_val.append(json_load[i]['hitCounter'])
                    mny.append(json_load[i]['collectedMoney'])
                    hlth.append(json_load[i]['collectedHealth'])
                    sndID.append(json_load[i]['idOfSound'])
                    sndTime.append(json_load[i]['timestampOfSound'])
                    hori_min.append(json_load[i]['xMin'])
                    vert_min.append(json_load[i]['yMin'])
                    hori_max.append(json_load[i]['xMax'])
                    vert_max.append(json_load[i]['yMax'])
                #Put the data in a dictionary with the SUB title 
                sub_data = {'x':hori_val,'y':vert_val,'timestamp':timestamp,'deathCount':death_val,'shootsCounter':shoot_val,'hitCounter':hit_val,'collectedMoney':mny,'collectedHealth':hlth,'idOfSound':sndID,'timestampOfSound':sndTime,'xMin':hori_min,'yMin':vert_min,'xMax':hori_max,'yMax':vert_max}
                data[sub_name] = sub_data

In [10]:
# function to check if the subject is moving,just ambulating, r is median rows before a change happens, 
#The high movement makes sense since that's what the game is all about
def check_move(smpl,r):
    x_prev = 0
    y_prev = 0
    x_min_prev = 0
    y_min_prev = 0
    x_max_prev = 0
    y_max_prev = 0
    mc_prev = 0
    sc_prev = 0
    for i in range(smpl.shape[0]):
        #Checking for position change
        if (smpl['x'][i] != x_prev or smpl['y'][i] != y_prev) and smpl['shootsCounter'][i] == sc_prev and smpl['collectedMoney'][i] == mc_prev:
            n = 0
            x_prev = smpl['x'][i]
            y_prev = smpl['y'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 1
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 1
                n+= 1

        elif (smpl['xMax'][i] != x_max_prev or smpl['yMax'][i] != y_max_prev) and smpl['shootsCounter'][i] == sc_prev  and smpl['collectedMoney'][i] == mc_prev:
            n = 0
            x_max_prev = smpl['xMax'][i]
            y_max_prev = smpl['yMax'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 1
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 1
                n+= 1

        elif (smpl['xMin'][i] != x_min_prev or smpl['yMin'][i] != y_min_prev) and smpl['shootsCounter'][i] == sc_prev and smpl['collectedMoney'][i] == mc_prev:
            n = 0
            x_min_prev = smpl['xMin'][i]
            y_min_prev = smpl['yMin'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 1
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 1
                n+= 1

        else:
            mc_prev = smpl['collectedMoney'][i]
            sc_prev = smpl['shootsCounter'][i]

In [11]:
#Function to check for moving and shooting powerups
def check_move_shoot(smpl,r):
    x_prev = 0
    y_prev = 0
    x_min_prev = 0
    y_min_prev = 0
    x_max_prev = 0
    y_max_prev = 0
    mc_prev = 0
    sc_prev = 0
    for i in range(smpl.shape[0]):
        #Checking for position change
        if (smpl['x'][i] != x_prev or smpl['y'][i] != y_prev) and smpl['shootsCounter'][i] != sc_prev and smpl['collectedMoney'][i] == mc_prev:
            n = 0
            x_prev = smpl['x'][i]
            y_prev = smpl['y'][i]
            sc_prev = smpl['shootsCounter'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 2
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 2
                n+= 1

        elif (smpl['xMax'][i] != x_max_prev or smpl['yMax'][i] != y_max_prev) and smpl['shootsCounter'][i] != sc_prev  and smpl['collectedMoney'][i] == mc_prev:
            n = 0
            x_max_prev = smpl['xMax'][i]
            y_max_prev = smpl['yMax'][i]
            sc_prev = smpl['shootsCounter'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 2
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 2
                n+= 1

        elif (smpl['xMin'][i] != x_min_prev or smpl['yMin'][i] != y_min_prev) and smpl['shootsCounter'][i] != sc_prev and smpl['collectedMoney'][i] == mc_prev:
            n = 0
            x_min_prev = smpl['xMin'][i]
            y_min_prev = smpl['yMin'][i]
            sc_prev = smpl['shootsCounter'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 2
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 2
                n+= 1

        else:
            sc_prev = smpl['shootsCounter'][i]
            mc_prev = smpl['collectedMoney'][i]

In [12]:
#Function to check if the player is moving to pick up items
def check_move_pick(smpl,r):
    x_prev = 0
    y_prev = 0
    x_min_prev = 0
    y_min_prev = 0
    x_max_prev = 0
    y_max_prev = 0
    mc_prev = 0
    hc_prev = 0
    sc_prev = 0
    for i in range(smpl.shape[0]):
        #Checking for position change
        if (smpl['x'][i] != x_prev or smpl['y'][i] != y_prev) and smpl['shootsCounter'][i] == sc_prev and smpl['collectedMoney'][i] != mc_prev:
            n = 0
            x_prev = smpl['x'][i]
            y_prev = smpl['y'][i]
            mc_prev = smpl['collectedMoney'][i]
            hc_prev = smpl['collectedHealth'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 3
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 3
                n+= 1

        elif (smpl['xMax'][i] != x_max_prev or smpl['yMax'][i] != y_max_prev) and smpl['shootsCounter'][i] == sc_prev  and smpl['collectedMoney'][i] != mc_prev:
            n = 0
            x_max_prev = smpl['xMax'][i]
            y_max_prev = smpl['yMax'][i]
            mc_prev = smpl['collectedMoney'][i]
            hc_prev = smpl['collectedHealth'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 3
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 3
                n+= 1

        elif (smpl['xMin'][i] != x_min_prev or smpl['yMin'][i] != y_min_prev) and smpl['shootsCounter'][i] == sc_prev and smpl['collectedMoney'][i] != mc_prev:
            n = 0
            x_min_prev = smpl['xMin'][i]
            y_min_prev = smpl['yMin'][i]
            mc_prev = smpl['collectedMoney'][i]
            hc_prev = smpl['collectedHealth'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 3
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 3
                n+= 1

        else:
            mc_prev = smpl['collectedMoney'][i]
            sc_prev = smpl['shootsCounter'][i]

In [13]:
#Moving Shooting and Pickup
def check_move_shoot_pick(smpl,r):
    x_prev = 0
    y_prev = 0
    x_min_prev = 0
    y_min_prev = 0
    x_max_prev = 0
    y_max_prev = 0
    mc_prev = 0
    sc_prev = 0
    for i in range(smpl.shape[0]):
        #Checking for position change
        if (smpl['x'][i] != x_prev or smpl['y'][i] != y_prev) and smpl['shootsCounter'][i] != sc_prev and smpl['collectedMoney'][i] != mc_prev:
            n = 0
            x_prev = smpl['x'][i]
            y_prev = smpl['y'][i]
            mc_prev = smpl['collectedMoney'][i]
            hc_prev = smpl['collectedHealth'][i]
            sc_prev = smpl['shootsCounter'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 4
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 4
                n+= 1

        elif (smpl['xMax'][i] != x_max_prev or smpl['yMax'][i] != y_max_prev) and smpl['shootsCounter'][i] != sc_prev  and smpl['collectedMoney'][i] != mc_prev:
            n = 0
            x_max_prev = smpl['xMax'][i]
            y_max_prev = smpl['yMax'][i]
            mc_prev = smpl['collectedMoney'][i]
            sc_prev = smpl['shootsCounter'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 4
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 4
                n+= 1

        elif (smpl['xMin'][i] != x_min_prev or smpl['yMin'][i] != y_min_prev) and smpl['shootsCounter'][i] != sc_prev and smpl['collectedMoney'][i] != mc_prev:
            n = 0
            x_min_prev = smpl['xMin'][i]
            y_min_prev = smpl['yMin'][i]
            mc_prev = smpl['collectedMoney'][i]
            sc_prev = smpl['shootsCounter'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 4
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 4
                n+= 1

In [14]:
def check_stand(smpl,r):
    x_prev = 0
    y_prev = 0
    x_min_prev = 0
    y_min_prev = 0
    x_max_prev = 0
    y_max_prev = 0
    sc_prev = 0
    for i in range(smpl.shape[0]):
        #Checking for position change
        if smpl['xMin'][i] == x_min_prev and smpl['yMin'][i] == y_min_prev and smpl['xMax'][i] == x_max_prev and smpl['yMax'][i] == y_max_prev and smpl['x'][i] == x_prev and smpl['y'][i] == y_prev:
            n = 0
            while n < r:#Putting the labels for all values above and below by 10 rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 5
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 5
                n+= 1
        else:
            x_prev = smpl['x'][i]
            y_prev = smpl['y'][i]
            x_min_prev = smpl['xMin'][i]
            y_min_prev = smpl['yMin'][i]
            x_max_prev = smpl['xMax'][i]
            y_max_prev = smpl['yMax'][i]


In [15]:
#Function to check for shooting
def check_shoot(smpl,r):
    #Labelling the data for attacking
    x_prev = 0
    y_prev = 0
    x_min_prev = 0
    y_min_prev = 0
    x_max_prev = 0
    y_max_prev = 0
    sc_prev = 0
    for i in range(smpl.shape[0]):
        #Checking for position change
        if smpl['xMin'][i] == x_min_prev and smpl['yMin'][i] == y_min_prev and smpl['xMax'][i] == x_max_prev and smpl['yMax'][i] == y_max_prev and smpl['x'][i] == x_prev and smpl['y'][i] == y_prev and smpl['shootsCounter'][i] != sc_prev:
            n = 0
            sc_prev = smpl['shootsCounter'][i]
            while n < r:#Putting the labels for all values above and below by 10 rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 6
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 6
                n+= 1
        else:
            x_prev = smpl['x'][i]
            y_prev = smpl['y'][i]
            x_min_prev = smpl['xMin'][i]
            y_min_prev = smpl['yMin'][i]
            x_max_prev = smpl['xMax'][i]
            y_max_prev = smpl['yMax'][i]


In [16]:
#Function to check how quickly do the values change
def change_val(frame):
    median_list = []
    for column in frame.columns:
        if 'timestamp' in column or 'Sound' in column or 'death' in column:
            print(column)
            continue
        curr_val = float('-inf')
        time = []
        for i in range(frame.shape[0]):
            if curr_val != frame[column][i]:
                curr_val = frame[column][i]
                time.append(frame['timestamp'][i])
        
        difference = np.diff(time)
        if len(difference) == 0:
            median_list.append(float('inf'))
        else:
            median_list.append(statistics.median(difference))
    print(median_list)
    return min(median_list)

In [17]:
import statistics
#Changing the data into Dataframes and adding activities
for key in data:
    smpl = data[key]
    sub_frame = pd.DataFrame(data = smpl)
    
    #Dropping DeathCount as it doesn't provide any new information
    sub_frame.drop(columns = ['deathCount'])
    
    #Finding the average time between the inputs and the number of rows to check before
    md = statistics.median(sub_frame['timestamp'].diff()[1:])
    change = change_val(sub_frame)
    radius = math.ceil(change/md)
    
    
    activity = [-1 for i in range(sub_frame.shape[0])]
    sub_frame["activityLabel"] = activity
    
    
    #Increasing the range of the move sub activities
    check_move(sub_frame,radius)
    check_move_shoot(sub_frame,4*radius)
    #Saw something and then shot or picked the item up, hence increasing the radius to capture that
    check_move_pick(sub_frame,4*radius)
    check_move_shoot_pick(sub_frame,8*radius) #More inputs needed
    check_stand(sub_frame,radius)
    check_shoot(sub_frame,2*radius)

    
    data[key] = sub_frame

#Double the limit for the item pickup

timestamp
deathCount
idOfSound
timestampOfSound
[116.0, 116, 217, 217, 2575.0, 9548, 116, 116.0, 116, 116.0]


/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smpl['activityLabel'][i+n] = 1
/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smpl['activityLabel'][i-n] = 1
/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

timestamp
deathCount
idOfSound
timestampOfSound
[116.0, 116, 218, 217, 3575.5, 22168, 116, 116.0, 116.0, 116.0]


/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smpl['activityLabel'][i+n] = 1
/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smpl['activityLabel'][i-n] = 1
/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

timestamp
deathCount
idOfSound
timestampOfSound
[116, 116, 934.0, 1016, 2550, 11250, 116, 116, 116, 116]


...

/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smpl['activityLabel'][i+n] = 1
/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smpl['activityLabel'][i-n] = 1
/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

timestamp
deathCount
idOfSound
timestampOfSound
[116.0, 116.0, 232, 218, 5033, 22818, 116.0, 116.0, 116.0, 116]


/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smpl['activityLabel'][i+n] = 1
/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smpl['activityLabel'][i-n] = 1
/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

timestamp
deathCount
idOfSound
timestampOfSound
[116, 116, 225.0, 217.0, 3091.5, 18334, 116, 116.0, 116, 116.0]


/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smpl['activityLabel'][i+n] = 1
/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smpl['activityLabel'][i-n] = 1
/var/folders/_y/7s4xs2ls4ys1mf_l07c83p680000gn/T/ipykernel_21554/3702941401.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

#Changing the data into Dataframes and adding activities
for key in data:
    smpl = data[key]
    sub_frame = pd.DataFrame(data = smpl)   
    data[key] = sub_frame

data['236']

In [21]:
print(data['200']['activityLabel'].value_counts())

1    839
5    435
4    365
6    149
3     94
2     82
Name: activityLabel, dtype: int64


In [25]:
#Check the overall class size and check the random chance of getting the activity right at the end
random_chance = []
for key in data:
    
    vals = data[key]['activityLabel'].value_counts()
    sum_num = sum(vals)
    list_num = [x/sum_num for x in vals]
    
    random_chance.append(max(list_num))

In [29]:
statistics.mean(random_chance)

0.574079510672603

In [30]:
statistics.median(random_chance)

0.5941629688353094

In [ ]:
#Collecting all the data and putting it in a timeseries format